# Prediction Data Cleaning
Input: 
> 瞬測儀Raw data

Output:
> true.npy : data after cleaning for prediction

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# below is for loading data from google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# put your folder path, ex: '/content/drive/MyDrive/.../'
folder_path = 'folderpath'

# put your data path, ex: '/content/drive/MyDrive/.../'
input_data_rate = '瞬測儀 raw data file path'
data_rate = pd.read_csv(input_data_rate, header=None)

print(data_rate.head())
print(data_rate.shape)

In [ ]:
"""
生成每筆資料之 產生檢驗單號時間_機台編號_工單號 作為編號
(編號格式為根據過去瞬測儀資料清洗後之編號)
"""
id = []
for i in range(len(data_rate)): # 逐筆資料
  id_no = ""
  date_init = str(data_rate[36].str[0:10][i]).split(' ')[0] # 取產生檢驗單號時間之日期
  time_init = str(data_rate[36][i]).split(' ',1)            # 取產生檢驗單號時間之時間

  # 將取得之日期改變格式 2022/1/1 -> 20220101
  if date_init != 'nan': 
    date_ini = datetime.strptime(date_init, "%Y/%m/%d")
    date_ini = date_ini.strftime("%Y%m%d")

  # 將取得之時間改變格式 下午02:46:20 -> 14:46
  if time_init[0] != 'nan' :
    time_ini = time_init[1][0:5] #小時
    dict1 = {'下午 01':13,'下午 02':14,'下午 03':15,'下午 04':16,'下午 05':17,'下午 06':18, \
              '下午 07':19,'下午 08':20,'下午 09': 21,'下午 10': 22,'下午 11': 23,'上午 12': 0, \
              '上午 01':1,'上午 02':2,'上午 03':3,'上午 04':4,'上午 05':5,'上午 06':6,'上午 07':7, \
              '上午 08':8,'上午 09': 9,'上午 10': 10,'上午 11': 11,'下午 12': 12}
    time_hour = int(pd.Series([time_ini]).map(dict1)) # 小時 12小時制轉換為24小時
    time_min = time_init[1][6:8] #分鐘

    time = str(time_hour)+' '+str(time_min)
    time = datetime.strptime(time, "%H %M")
    time = time.strftime("%H:%M")

    machine = str(data_rate[10][i]) #機台編號
    work = str(data_rate[8][i])     #工單號

    id_no = date_ini + time +'_'+machine+'_'+work #產生檢驗單號時間_機台編號_工單號 作為編號

  id.append(id_no)

data_rate["id"] = id
print(id)

In [ ]:
"""
產生僅具編號及實際測量值之檔案
"""
# 只取特定料號資料
data_rate = data_rate[data_rate[5] == "0162B00100"]

data = pd.DataFrame()
data["id"] = data_rate["id"] #編號

for col_num in range(14,27):
  data[str(col_num-13)] = data_rate[col_num] #各實際測量值

np.save(folder_name+'true.npy', data) 
print(data)